In [1]:
import PyPDF2
import tabula
import pandas as pd
import warnings
import pdfplumber
import datefinder
from datetime import datetime
import re
warnings.filterwarnings('ignore')

In [2]:
cities = set(["NEW YORK", "AMSTERDAM", "MEXICO CITY", "SYDNEY", "MUMBAI", "TOKYO", "Toronto", "Mumbai", "India", "New York", "Sydney", "Tokyo", "Amsterdam", "Mexico City"])
months = set(["JANUARY", "FEBRUARY", "MARCH", "APRIL", "MAY", "JUNE", "JULY", "AUGUST", "SEPTEMBER", "OCTOBER", "NOVEMBER", "DECEMBER", "January", "February","March","April","May","June","July","August","September","October","November","December"])

In [23]:
pdf_list = ["/Users/lukegeel/Desktop/Rebalances/s&p TSX/s&p TSX composite after 2018/s&p TSX composite june-24-19.pdf",
           #"/Users/lukegeel/Desktop/Rebalances/s&p TSX/s&p TSX composite before 2018/s&p TSX composite dec-14-12.pdf",
           "/Users/lukegeel/Desktop/Rebalances/s&p TSX/s&p TSX dividend/S&P:TSX Dividend may-25-20.pdf",
           "/Users/lukegeel/Desktop/Rebalances/s&p TSX/s&p TSX Preferred/before 2017/s&pTSX preferred apr-2-15.pdf",
           "/Users/lukegeel/Desktop/Rebalances/s&p TSX/s&p TSX Preferred/s&pTSX Preferred july-5-19.pdf",
           "/Users/lukegeel/Desktop/Rebalances/s&p Topix/s&p topix jun-8-18.pdf",
           "/Users/lukegeel/Desktop/Rebalances/s&p Topix/before 2016/s&p Topix sept-11-14.pdf",
           "/Users/lukegeel/Desktop/Rebalances/s&p NZX starting 2015/s&pNZX dec-11-20.pdf",
           "/Users/lukegeel/Desktop/Rebalances/s&p NZX starting 2015/before 2017/s&pNZX june-10-16.pdf",
           "/Users/lukegeel/Desktop/Rebalances/s&p Latin America/s&p latin america bmv china sept-11-20.pdf",
           "/Users/lukegeel/Desktop/Rebalances/s&p Dow Jones U.S. Select/Rebalance/before 2020/s&p Dow Jones US Select Mar18.pdf",
           "/Users/lukegeel/Desktop/Rebalances/s&p Dow Jones U.S. Select/Rebalance/after 2020/s&p Dow Jones US select Mar21.pdf",
           #"/Users/lukegeel/Desktop/Rebalances/s&p Dow Jones U.S. Select/Constitute change/s&p dow jones US select may21.pdf",
           "/Users/lukegeel/Desktop/Rebalances/s&p BSE/reconstitution/s&pBSE jun-12-15.pdf",
           "/Users/lukegeel/Desktop/Rebalances/s&p BSE/changes/s&pBSE dec-19-19.pdf",
           "/Users/lukegeel/Desktop/Rebalances/s&p ASX starting at 2017/s&pASX june-3-22.pdf",
           "/Users/lukegeel/Desktop/Rebalances/s&p ASX starting at 2017/s&p ASX before 2017/s&pASX mar-11-16.pdf",
           "/Users/lukegeel/Desktop/Rebalances/s&p 500/s&p500 dividend/s&p500 dividend aristicrats jan22.pdf",
           "/Users/lukegeel/Desktop/Rebalances/s&p 500/s&p500 before july-21-2020/s&p500 april-16-2018.pdf",
           "/Users/lukegeel/Desktop/Rebalances/s&p 500/s&p500 after july-21-2020/s&p500 apr-5-2021.pdf"       
            ]

In [51]:
def tokyo_before2016(pdf_path, market, release_date):
    final_result = pd.DataFrame()   
    tables = tabula.read_pdf(
        pdf_path,
        pages='all',
        lattice=True,
        multiple_tables=True,
    )

    for df in tables:
    
        if "–" in df.columns[0]:
            first_header = df.columns[0]
            split_result = first_header.split('–')
            if 'After Market' in  split_result:
                part_a, part_b = split_result[1].split('After Market')
            else:
                pass
            df.columns = ["Action", "Exchange Code", "Company"]
            df = df.drop(0)
            df['Index'] = split_result[0]
            df['Market'] = market
            df['Release date'] = release_date
            df['Effective date'] = effective_date
            final_result = pd.concat([final_result, df])

        elif len(df.columns) < 3 and len(df.columns) > 1:
            new_row = {
                df.columns[0]: df.columns[0],
                df.columns[1]: None,
                df.columns[2]: None
            }
            df.loc[-1] = new_row
            df.columns = ['Action', 'Exchange Code', 'Company']
            df['Index'] = split_result[0]
            df['Market'] = market
            df['Release date'] = release_date
            df['Effective date'] = part_b
            row_to_move = df.loc[[-1], :]
            df = pd.concat([row_to_move, df.drop(index=-1)])
            final_result = pd.concat([final_result, df])

        print(final_result)
        
def tokyo_after2016(pdf_path, market, release_date):
    final_result = pd.DataFrame()   
    tables = tabula.read_pdf(
        pdf_path, 
        pages='all', 
        lattice=True, 
        multiple_tables=True)
    
    result = pd.DataFrame()
    previous_table = pd.DataFrame()
    prev_col = None
    first_header = None
    split_result = None
    Index = None
    Effective_date = None
    final_previous_table = pd.DataFrame()

    for df in tables:
        if "–" in df.columns[0]:
            first_header = df.columns[0]
            split_result = first_header.split('–')
            part_a, part_b = split_result[1].split('Open on')
            df.columns = ["Action", "Code", "Company"]
            df = df.drop(0)
            df['Index'] = split_result[0]
            df['Market'] = market
            df['Release date'] = release_date
            df['Effective date'] = part_b
            result = pd.concat([result, df])
            previous_table = df

        elif "–" not in df.columns[0]:
            new_row = {df.columns[0]: df.columns[0], df.columns[1]: df.columns[1], df.columns[2]: df.columns[2]}
            df.loc[-1] = new_row
            df.columns = ['Action', 'Code', 'Company']
            df['Index'] = Index
            df['Market'] = market
            df['Release date'] = release_date
            df['Effective date'] = part_b
            row_to_move = df.loc[[-1], :]
            df = pd.concat([row_to_move, df.drop(index=-1)])
            result = pd.concat([result, df])

    final_result = pd.concat([final_result, result])
    print(final_result)
    

def mumbai(pdf_path, market, release_date):
    final_result = pd.DataFrame()   
    tables = tabula.read_pdf(
    pdf_path, 
    pages='all', 
    lattice=True, 
    multiple_tables=True)
    
    for table in tables:
    
        action_column_count = sum(1 for col in table.columns if col.lower() == 'drop' or col.lower() == 'add')
        index_column = next((col for col in table.columns if col.lower() == 'index' or col.lower() == 'indices'), None)

        if action_column_count == 2 and index_column != None:

            table = table.rename(columns={table.columns[1]: "Drop Exchange Ticker", table.columns[2]: "Drop Stock Name", table.columns[3]: "Add Exchange Ticker", table.columns[4]: "Add Stock Name"})
            prev_date = None

            for i, row in table.iterrows():
                    if pd.isna(row['Unnamed: 2']):
                        table.loc[i, 'Unnamed: 2'] = prev_date
                    else:
                        prev_date = row['Unnamed: 2']

            droptable = pd.concat([index_column, table['Drop Exchange Ticker'], table['Drop Stock Name'], table['Unnamed: 2']], axis=1)
            droptable['Release date'] = release_date
            droptable = droptable.rename(columns={droptable.columns[1]:"Exchange ticker"})
            droptable = droptable.rename(columns={droptable.columns[2]:"Stock Name"})
            droptable = droptable.rename(columns={droptable.columns[3]:"Effective date"})
            droptable = droptable.drop(0)
            droptable = droptable[droptable != '--'].dropna()

            addtable = pd.concat([index_column, table['Add Exchange Ticker'], table['Add Stock Name'], table['Unnamed: 2']], axis=1)
            addtable['Release date'] = release_date
            addtable = addtable.rename(columns={addtable.columns[1]:"Exchange ticker"})
            addtable = addtable.rename(columns={addtable.columns[2]:"Stock Name"})
            addtable = addtable.rename(columns={addtable.columns[3]:"Effective date"})
            addtable = addtable.drop(0)
            addtable = addtable[addtable != '--'].dropna()

            combined_table = pd.concat([droptable, addtable], axis=0)
            print(combined_table)

        if action_column_count == 2 and index_column == None:

            table = table.rename(columns={table.columns[0]: "Drop Exchange Ticker", table.columns[1]: "Drop Stock Name", table.columns[2]: "Add Exchange Ticker", table.columns[3]: "Add Stock Name"})
            prev_date = None


            droptable = pd.concat([table['Drop Exchange Ticker'], table['Drop Stock Name']], axis=1)
            action = droptable.columns[0]
            droptable['Release date'] = release_date
            droptable = droptable.rename(columns={droptable.columns[0]:"Exchange ticker"})
            droptable = droptable.rename(columns={droptable.columns[1]:"Stock Name"})
            droptable['Action'] = action
            droptable = droptable.drop(0)
            droptable = droptable[droptable != '--'].dropna()

            addtable = pd.concat([table['Add Exchange Ticker'], table['Add Stock Name']], axis=1)
            action = addtable.columns[0]
            addtable['Release date'] = release_date
            addtable = addtable.rename(columns={addtable.columns[0]:"Exchange ticker"})
            addtable = addtable.rename(columns={addtable.columns[1]:"Stock Name"})
            addtable['Action'] = action
            addtable = addtable.drop(0)
            addtable = addtable[addtable != '--'].dropna()

            combined_table = pd.concat([droptable, addtable], axis=0)
            combined_table['Action'] = combined_table['Action'].str.replace('Exchange Ticker', '')
            print(combined_table)

        elif action_column_count == 1:
            action_column = next((col for col in table.columns if col.lower() == 'drop' or col.lower() == 'add'), None)

            prev_date = None
            prev_ticker = None
            prev_company = None
            prev_index = None

            for i, row in table.iterrows():
                if pd.isna(row['Unnamed: 1']):
                    table.loc[i, 'Unnamed: 1'] = prev_date
                else:
                    prev_date = row['Unnamed: 1']

                if pd.isna(action_column):
                    table.loc[i, action_column] = prev_ticker
                else:
                    prev_ticker = action_column

                if pd.isna(row['Unnamed: 0']):
                    table.loc[i, 'Unnamed: 0'] = prev_company
                else:
                    prev_company = row['Unnamed: 0']

                if pd.isna(row[index_column]):
                    table.loc[i, index_column] = prev_index
                else:
                    prev_index = row[index_column]

            action = table.columns[1]
            table['Action'] = action
            table['Release date'] = release_date
            table = table.rename(columns={table.columns[1]: "Exchange Ticker", table.columns[2]: "Stock Name", table.columns[3]: "Effective Date"})
            table = table.drop(table.index[0])

            print(table)
            pass

def mexicocity(pdf_path, market, release_date):
    final_result = pd.DataFrame()   
    page_number = 0  # Specify the page number (starting from 0)
    x1, y1, x2, y2 = 1, 190, 595, 280  # Specify the coordinates of the area (in points)

    with pdfplumber.open(pdf_path) as pdf:
        page = pdf.pages[page_number]

        text_content = page.crop((x1, y1, x2, y2)).extract_text()

    matches = datefinder.find_dates(text_content)

    pattern = r"\b(?:\d{1,2}[-/ ]\d{1,2}[-/ ]\d{2,4}|\w+ \d{1,2},? \d{4})\b"

    regex_matches = re.findall(pattern, text_content) # Find all matches of the date pattern in the text

    regex_matches = [datetime.strptime(date, "%B %d, %Y") for date in regex_matches] # Convert regex matches to datetime objects

    all_matches = list(matches) + regex_matches # Combine the matches from both datefinder and regex

    unique_matches = list(set(all_matches)) # Remove duplicates by converting the list to a set and back to a list

    sorted_matches = sorted(unique_matches) # Sort the unique matches in ascending order

    min_difference = None # Find the two nearest dates
    nearest_dates = []

    for i in range(len(sorted_matches) - 1):
        difference = sorted_matches[i + 1] - sorted_matches[i]
        if min_difference is None or difference < min_difference:
            min_difference = difference
            nearest_dates = [sorted_matches[i], sorted_matches[i + 1]]

    nearest_dates.sort() # Sort the nearest dates in ascending order

    effective_date = nearest_dates[-1]

    tables = tabula.read_pdf(
        pdf_path, 
        pages='all', 
        lattice=True, 
        multiple_tables=True, 
    )
    df = tables[0]
    df = df.drop(columns=['Unnamed: 2', 'Action', 'Company Name', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5'])
    df = df.rename(columns={'Ticker': 'Company Name', 'Unnamed: 1': 'Action', 'Unnamed: 0': 'Ticker'})
    df['Market'] = market
    df['Release date'] = release_date
    df['Effective date'] = effective_date

    final_result = pd.concat([final_result, df])
    print(final_result)
    
def nyse_before72120(pdf_path, market, release_date):
    final_result = pd.DataFrame()   
    tables = tabula.read_pdf(
        pdf_path, 
        pages='all', 
        lattice=True, 
        multiple_tables=True, 
    )
    result = pd.DataFrame()
    
    for i, df in enumerate(tables):
        if df.empty:
            continue

        df = df.rename(columns={'Unnamed: 0': df['Unnamed: 0'][0], 'Unnamed: 1': df['Unnamed: 1'][0], 'Unnamed: 2': df['Unnamed: 2'][0]})
        df = df.drop(0)

        if df.columns[2] == "GICS ECONOMIC\rSECTOR":
            df.rename(columns={"GICS ECONOMIC\rSECTOR": "GICS ECONOMIC SECTOR"}, inplace=True)

        first_header = df.columns[0]
        split_result = first_header.split('–')

        df['Index'] = split_result[0]
        df['Market'] = market
        df['GICS SUB-INDUSTRY'] = df['GICS SUB-INDUSTRY'].str.replace("\r", " ")
        df['GICS ECONOMIC SECTOR'] = df['GICS ECONOMIC SECTOR'].str.replace("\r", " ")
        df['COMPANY'] = df['COMPANY'].str.replace("\r", " ")
        df['Release date'] = release_date
        df['Effective date'] = split_result[1]
        df = df.rename(columns={first_header: 'Action'})
        df['Action'].fillna(method='ffill', inplace=True)

        result = pd.concat([result, df])

    final_result = pd.concat([final_result, result])
    print(final_result)

def nyse_after72120(pdf_path, market, release_date):
    final_result = pd.DataFrame()   
    tables = tabula.read_pdf(
        pdf_path, 
        pages='all', 
        lattice=False, 
        multiple_tables=True)
    result = pd.DataFrame()
    prev_date = None
    prev_index = None
    prev_sector = None
    prev_company = None
    for i, df in enumerate(tables):
        df['Market'] = market
        df['Release date'] = release_date
        if 'Action' not in df.columns:
            continue
        if df.columns[0] == "Effective":
            df = df.rename(columns={'Effective': 'Effective Date'})
            df = df.drop(0)
        else:
            pass
        
        if df.columns[3] == "Company":
            df = df.rename(columns={'Company': 'Company Name'})
            df = df.drop(0)
        else:
            pass
        
        if df.columns[0] == "Unnamed: 0":
            move_index_name = df.columns[1]
            move_action_name = df.columns[2]
            move_company_name = df.columns[3]
            move_ticker_name = df.columns[4]
            move_sector_name = df.columns[5]
            
            df = df.rename(columns={df.columns[0]: 'Effective Date'}) 
            df = df.rename(columns={df.columns[1]: 'Index Name'})
            df = df.rename(columns={df.columns[2]: 'Action'})
            df = df.rename(columns={df.columns[3]: 'Company Name'})
            df = df.rename(columns={df.columns[4]: 'Ticker'})
            df = df.rename(columns={df.columns[5]: 'GICS Sector'})
            
            new_row = {
                'Effective Date' : 'na',
                'Index Name': move_index_name,
                'Action': move_action_name,
                'Company Name': move_company_name,
                'Ticker': move_ticker_name,
                'GICS Sector': move_sector_name
            }
            
            df = df.append(new_row, ignore_index=True)
            
        else:
            pass        
            
        for i, row in df.iterrows():
            if pd.isna(row['Effective Date']):
                df.at[i, 'Effective Date'] = prev_date
            else:
                prev_date = row['Effective Date']

        for i, row in df.iterrows():       
            if pd.isna(row['Index Name']):
                df.at[i, 'Index Name'] = prev_index
            else:
                prev_index = row['Index Name']

        for i, row in df.iterrows():
            if pd.isna(row['Action']):
                if pd.notna(row['Company Name']):
                    df.loc[i-1, 'Company Name'] = prev_company + " " + row['Company Name']
                elif pd.notna(row['GICS Sector']):
                    df.loc[i-1, 'GICS Sector'] = prev_sector + " " + row['GICS Sector']
                df = df.dropna()
            else:
                prev_company = row['Company Name']
                prev_index = row['Index Name']
                prev_sector = row['GICS Sector']

        result = pd.concat([result, df])

    final_result = pd.concat([final_result, result])
    
    final_result = final_result.dropna(subset=['Release date'])       

    print(final_result)

def tsxpreferred_after112017(pdf_path, market, release_date):
    final_result = pd.DataFrame()   
    tables = tabula.read_pdf(
    pdf_path, 
    pages='all', 
    lattice=True, 
    multiple_tables=True)

    for table in tables:
        table['Index'], table['Action'] = table.columns[0].split('-')
        table['Market'] = market
        table['Release date'] = release_date
        table.rename(columns={table.columns[0]: 'symbol', table.columns[1]: 'company name'}, inplace=True)
        table = table.iloc[1:]
        print(table)

def tsxpreferred_before112017(pdf_path, market, release_date):
    final_result = pd.DataFrame()   
    page_number = 0  # Specify the page number (starting from 0)
    x1, y1, x2, y2 = 1, 190, 595, 280  # Specify the coordinates of the area (in points)

    with pdfplumber.open(pdf_path) as pdf: # Open the PDF file
        page = pdf.pages[page_number]

        text_content = page.crop((x1, y1, x2, y2)).extract_text() # Extract text from the specified area

    matches = datefinder.find_dates(text_content)

    pattern = r"\b(?:\d{1,2}[-/ ]\d{1,2}[-/ ]\d{2,4}|\w+ \d{1,2},? \d{4})\b" # Create a regular expression pattern to match date formats

    regex_matches = re.findall(pattern, text_content) # Find all matches of the date pattern in the text

    regex_matches = [datetime.strptime(date, "%B %d, %Y") for date in regex_matches] # Convert regex matches to datetime objects

    all_matches = list(matches) + regex_matches # Combine the matches from both datefinder and regex

    unique_matches = list(set(all_matches)) # Remove duplicates by converting the list to a set and back to a list

    sorted_matches = sorted(unique_matches) # Sort the unique matches in ascending order

    min_difference = None # Find the two nearest dates
    nearest_dates = []

    for i in range(len(sorted_matches) - 1):
        difference = sorted_matches[i + 1] - sorted_matches[i]
        if min_difference is None or difference < min_difference:
            min_difference = difference
            nearest_dates = [sorted_matches[i], sorted_matches[i + 1]]

    nearest_dates.sort() # Sort the nearest dates in ascending order

    effective_date = nearest_dates[-1]
    
    tables = tabula.read_pdf(
    pdf_path, 
    pages='all', 
    lattice=True, 
    multiple_tables=True, 
    )

    filtered_tables = [table for table in tables if 'McGRAW-HILL' not in table.columns]

    for table in filtered_tables:
        if table.shape[1] == 1:
            continue  # Ignore the one-column table and move to the next one

        table.columns = ['Symbol', 'Issue Name', 'CUSIP'] + list(table.columns[3:])

        table['Action'] = 'Addition'

        deletions_index = table[table[table.columns[0]] == 'DELETIONS'].index[0]
        table.loc[deletions_index:, 'Action'] = 'Deletion'

        table['Release date'] = release_date
        table['Market'] = market
        table['Effetive date'] = effective_date

        table.dropna(axis=0, how='any', inplace=True)  # Drop rows with NaN values
        table = table[table['Symbol'] != 'Symbol']

        print(table)

def tsxcomposite_after112017(pdf_path, market, release_date):
    final_result = pd.DataFrame()   
    tables = tabula.read_pdf(
        pdf_path, 
        pages='all', 
        lattice=True, 
        multiple_tables=True, 
    )
    result = pd.DataFrame()

    for df in tables:
        if df.empty:
            continue

        df = df.rename(columns={'Unnamed: 0': df['Unnamed: 0'][0], 'Unnamed: 1': df['Unnamed: 1'][0], 'Unnamed: 2': df['Unnamed: 2'][0]})
        df = df.drop(0)
        if df.columns[2] == "GICS SECTOR":
            df.rename(columns={"GICS\rSECTOR": "GICS SECTOR"}, inplace=True)

        first_header = df.columns[0]
        split_result = first_header.split('–')
        df['Index'] = split_result[0]
        df['Market'] = market
        df['GICS Sub-Industry'] = df['GICS SUB-INDUSTRY'].str.replace("\r", " ")
        if 'GICS\rSECTOR' in df.columns:
            df['GICS Sector'] = df['GICS\rSECTOR'].str.replace("\r", " ")
        df['Company'] = df['COMPANY'].str.replace("\r", " ")
        df['Release date'] = release_date
        df['Effective date'] = split_result[1]
        df = df.rename(columns={first_header: 'Action'})
        df['Stock Ticker'] = df['COMPANY'].str.extract(r'\((.*?)\)')
        df = df.drop(['COMPANY', 'GICS SUB-INDUSTRY'], axis=1)
        result = pd.concat([result, df])

    final_result = pd.concat([final_result, result])
    print(final_result)

def tsxcomposite_before112017(pdf_path, market, release_date):
    tables = tabula.read_pdf(
        pdf_path, 
        pages='all', 
        lattice=True, 
        multiple_tables=True, 
    )
    result = pd.DataFrame()

    for i, df in enumerate(tables):
            if df.empty:
                pass
            else:
                df = df.rename(columns={'Unnamed: 0':df['Unnamed: 0'][0], 'Unnamed: 1':df['Unnamed: 1'][0],'Unnamed: 2':df['Unnamed: 2'][0]})
                df = df.drop(0)
                if df.columns[2] == "GICS SECTOR":
                    df.rename(columns={"GICS SECTOR": "GICS SECTOR"}, inplace=True)
                else:
                    pass
                first_header = df.columns[0]
                split_result = first_header.split('–')
                df['Index'] = split_result[0]
                df['Market'] = market
                df['GICS SUB-INDUSTRY'] = df['GICS SUB-INDUSTRY'].apply(lambda x: x.replace("\r", " "))
                df['GICS SECTOR'] = df['GICS SECTOR'].apply(lambda x: x.replace("\r", " "))
                df['COMPANY'] = df['COMPANY'].apply(lambda x: x.replace("\r", " "))
                df['Release date'] = release_date
                df['Effective date'] = split_result[1]
                df = df.rename(columns={first_header:'Action'})
                df['Action'].fillna(method='ffill', inplace=True)
                result = pd.concat([result, df])

            final_result = pd.concat([final_result, result])
    print(final_result)
    
def sydney_before2017(pdf_path, market, release):
    final_result = pd.DataFrame()
    tables = tabula.read_pdf(
        pdf_path,
        pages='all',
        lattice=True,
        multiple_tables=True,
    )
    result = pd.DataFrame()
    
    for i, df in enumerate(tables):
        if len(df.columns) == 1:
            continue
        
        if "–" in df.columns[0]:
            first_header = df.columns[0]
            split_result = first_header.split('–')
            part_a, part_b = split_result[1].split('After')
            df = df.rename(columns={df.columns[0]: "Action"})
            df = df.rename(columns={'Unnamed: 0':"Code", 'Unnamed: 1':"Company"})
            df = df.drop(0)
            Index = split_result[0]
            df['Index'] = split_result[0]
            df['Market'] = market
            df['Release date'] = release_date
            df['Effective date'] = part_a
            result = pd.concat([result, df])
            previous_table = df

        elif "–" not in df.columns[0]: 
            new_row = {df.columns[0]: df.columns[0], df.columns[1]: df.columns[1], df.columns[2]: df.columns[2]}
            df.loc[-1] = new_row
            df.columns = ['Action', 'Code', 'Company']
            df['Index'] = Index
            df['Market'] = market
            df['Release date'] = release_date
            df['Effective date'] = part_a
            row_to_move = df.loc[[-1], :]
            df = pd.concat([row_to_move, df.drop(index=-1)])
            result = pd.concat([result, df])

    final_result = pd.concat([final_result, result])
    print(final_result) 
    
def sydney_after2017(pdf_path, market, release):
    final_result = pd.DataFrame()
    tables = tabula.read_pdf(
        pdf_path,
        pages='all',
        lattice=True,
        multiple_tables=True,
    )
    result = pd.DataFrame()
    for i, df in enumerate(tables):
        if "–" in df.columns[0]:
            first_header = df.columns[0]
            split_result = first_header.split('–')
            part_a, part_b = split_result[1].split('on')
            df = df.rename(columns={df.columns[0]: "Action", 'Unnamed: 0': "Code", 'Unnamed: 1': "Company"})
            df = df.drop(0)
            df['Index'] = split_result[0]
            df['Market'] = market
            df['Release date'] = release_date
            df['Effective date'] = part_b
            result = pd.concat([result, df])
        elif "–" not in df.columns[0]:
            new_row = {
                df.columns[0]: df.columns[0],
                df.columns[1]: df.columns[1],
                df.columns[2]: df.columns[2]
            }
            df.loc[-1] = new_row
            df.columns = ['Action', 'Code', 'Company']
            df['Index'] = split_result[0]
            df['Market'] = market
            df['Release date'] = release_date
            df['Effective date'] = part_b
            row_to_move = df.loc[[-1], :]
            df = pd.concat([row_to_move, df.drop(index=-1)])
            result = pd.concat([result, df])

    final_result = pd.concat([final_result, result])
    print(final_result)
    
def usselect_before2020(pdf_path, market, release_date):
    tables = tabula.read_pdf(
        pdf_path, 
        pages='all', 
        lattice=True, 
        multiple_tables=True 
    )
    table = tables[0]
    table = table.drop(columns = ['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 3', 'Unnamed: 5'])
    Index = table.columns[0]
    table['Index'] = Index
    ed = table.iloc[1,0]
    for month in months:
        if month in ed:
            first_part, second_part = ed.split(month)
            Effective_date = month + second_part 
    table['Effective date'] = Effective_date
    table['Release date'] = release_date
    table = table.drop(0)
    table = table.drop(1)
    table = table.drop(2)
    table = table.drop(3)
    table = table.drop(4)
    table = table.rename(columns={'Unnamed: 2':"Company", 'Unnamed: 4':"Action", 'Dow Jones U.S. Select Dividend Index':"Ticker"})
    table.dropna(inplace=True)
    table = table.reset_index(drop=True)
    print(table)
    
def usselect_after2020(pdf_path, market, release_date):
    tables = tabula.read_pdf(
        pdf_path, 
        pages='all', 
        lattice=True, 
        multiple_tables=True 
    )
    table = tables[0]
    table = table.drop(columns = ['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3'])
    table['Release date'] = release_date
    table['Market'] = market
    table['Index'] = 'Dow Jones U.S. Select Dividend Index'

    print(table)
    
def dividendaristocrat(pdf_path, market, release_date):
    tables = tabula.read_pdf(
        pdf_path, 
        pages='all', 
        lattice=True, 
        multiple_tables=True, 
    )

    result = pd.DataFrame()
    for i, df in enumerate(tables):
        if 'Unnamed: 0' in df.columns:
            df = df.drop(columns=['Unnamed: 0'])
        else:
            pass

        df['Market'] = market
        df['Release date'] = release_date
        df['Effective date'] = effective_date

        if i == 0:
            df['Action'] = 'Addition'
        else:
            df['Action'] = 'Deletion'
        result = pd.concat([result, df])

    print(result)

def tsxdividendaristocrat(pdf_path, market, release_date):
    tables = tabula.read_pdf(
    pdf_path, 
    pages='all', 
    lattice=True, 
    multiple_tables=True, 
    )

    result = pd.DataFrame()
    previous_table = pd.DataFrame()
    first_header = None
    Index = None

    for i, df in enumerate(tables):
        if "–" in df.columns[0]:
            if not previous_table.empty:
                result = result.append(previous_table, ignore_index=True)
                previous_table = pd.DataFrame()

            first_header = df.columns[0]
            split_result = first_header.split('–')
            df = df.rename(columns={df.columns[0]: "Action"})
            df = df.rename(columns={'Unnamed: 0': "Code", 'Unnamed: 1': "Company"})
            df = df.drop(0)
            Index = split_result[0]
            df['Index'] = Index
            df['Market'] = market
            df['Release date'] = release_date
            df['Effective date'] = split_result[1]
            result = result.append(df, ignore_index=True)

        elif "–" not in df.columns[0] and not df.empty:
            df.columns = ['Action', 'Code', 'Company']
            df['Market'] = market
            df['Index'] = Index
            df['Release date'] = release_date
            previous_table = previous_table.append(df, ignore_index=True)

    if not previous_table.empty:
        result = result.append(previous_table, ignore_index=True)

    final_result = result.reset_index(drop=True)
    print(final_result)

In [52]:
page_number = 0  # Specify the page number (starting from 0)
x1, y1, x2, y2 = 20, 100, 320, 280  # Specify the coordinates of the area (in points)

for pdf_path in pdf_list: # Open the PDF file
    city_found = False
    with pdfplumber.open(pdf_path) as pdf_plumbed:
        page = pdf_plumbed.pages[page_number]

        text_content = page.crop((x1, y1, x2, y2)).extract_text()
        
        for city in cities:
            if city in text_content:
                city_found = True
                market = city
            if city_found == True:
                break
        else:
            print("No city found in pdf:", pdf_path)
    
        matches = datefinder.find_dates(text_content)
        pattern = r"\b(?:\d{1,2}[-/ ]\d{1,2}[-/ ]\d{2,4}|\w+ \d{1,2},? \d{4})\b" # Create a regular expression pattern to match date formats
        regex_matches = re.findall(pattern, text_content) # Find all matches of the date pattern in the text
        
        regex_matches = [datetime.strptime(date, "%B %d, %Y") for date in regex_matches] # Convert regex matches to datetime objects
        all_matches = list(matches) + regex_matches # Combine the matches from both datefinder and regex
        
        unique_matches = list(set(all_matches)) # Remove duplicates by converting the list to a set and back to a list
        sorted_matches = sorted(unique_matches) # Sort the unique matches in ascending order
        
        sorted_matches = [date for date in sorted_matches if date.year >= 1000]
        sorted_matches = [date for date in sorted_matches if date.year <= 2049]
        
        release_date = sorted_matches[0]
        effective_date = sorted_matches[-1]

        market = market.lower()  # Convert the market to lowercase

        if market == 'tokyo':
            if release_date < datetime(2016, 1, 1):
                tokyo_before2016(pdf_path, market, release_date)
            else:
                tokyo_after2016(pdf_path, market, release_date)

        elif market == 'mexico city':
            mexicocity(pdf_path, market, release_date)

        elif market == 'sydney':
            if release_date < datetime(2017, 1, 1):
                sydney_before2017(pdf_path, market, release_date)
            else:
                sydney_after2017(pdf_path, market, release_date)

        elif market == 'mumbai' or market == 'india':
            mumbai(pdf_path, market, release_date)
        
        elif market == 'new york':
            
            if 'U.S. Select' in text_content:
                if release_date < datetime(2020, 7, 21):
                    usselect_before2020(pdf_path, market, release_date)
                else:
                    usselect_after2020(pdf_path, market, release_date)
            elif 'Dividend Aristocrats' in text_content:
                dividendaristocrat(pdf_path, market, release_date)
            else:
                
                if release_date < datetime(2020, 7, 21):
                    nyse_before72120(pdf_path, market, release_date)
                else:
                    nyse_after72120(pdf_path, market, release_date)

        elif market == 'toronto':
            if 'preferred' in text_content.lower():
                if release_date < datetime(2017, 1, 1):
                    tsxpreferred_before112017(pdf_path, market, release_date)
                else:
                    tsxpreferred_after112017(pdf_path, market, release_date)
                    
            elif 'aristocrats' in text_content.lower():
                tsxdividendaristocrat(pdf_path, market, release_date)
                
            elif 'composite' in text_content.lower():
                if release_date < datetime(2018, 1, 1):
                    continue
                else:
                    tsxcomposite_after112017(pdf_path, market, release_date)


    Action              GICS SECTOR                     Index   Market  \
1    ADDED                   Energy  S&P/TSX COMPOSITE INDEX   toronto   
2  DELETED                   Energy  S&P/TSX COMPOSITE INDEX   toronto   
3  DELETED                Materials  S&P/TSX COMPOSITE INDEX   toronto   
4  DELETED                Materials  S&P/TSX COMPOSITE INDEX   toronto   
5  DELETED  Consumer\rDiscretionary  S&P/TSX COMPOSITE INDEX   toronto   

                    GICS Sub-Industry                                Company  \
1  Oil & Gas Exploration & Production  Frontera Energy Corporation (TSX:FEC)   
2      Oil & Gas Equipment & Services   CES Energy Solutions Corp. (TSX:CEU)   
3                              Silver    Fortuna Silver Mines Inc. (TSX:FVI)   
4                                Gold                 New Gold Inc (TSX:NGD)   
5                        Distributors               Uni-Select Inc (TSX:UNS)   

  Release date  Effective date Stock Ticker  
1   2019-06-14   June 24, 20